# Projeto 1 - Ciência dos Dados

Nome: Gustavo Katz
Nome: _____

Nome: _____

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [35]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [36]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Gustavo\Documents\INSPER2\Cdados\Nova pasta\P1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [78]:
filename = 'PS5.xlsx'

In [79]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificacao,0: Não relevante\n1: Relevante\n2: Muito relevante
0,"fala galera, livezinha no rainbow, vem ai dar ...",1,NaN
1,atualizações\n\nxbox series x : 10.3gb\nxbox o...,2,NaN
2,– season 6 resumo de tamanhos!\n\n– xbox: 26 g...,2,NaN
3,@loud_voltan vai jogar ps5 né veia? \n\nbom de...,1,NaN
4,@ocervejeiro_ ps5 pro com certeza. https://t.c...,1,NaN


In [80]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,classificacao2,0: Não relevante\n1: Relevante\n2: Muito relevante
0,"@micalibruno apesar de ser um game de 2018, oc...",1,NaN
1,@ileohhh @fnstatusbr @brasil_fortnite no ps5 a...,1,NaN
2,rt @tiagohardco: ontem eu tava jogando pillars...,1,NaN
3,@heyzubshere pois é kfhskdsdfsdf e o ps5 tá da...,1,NaN
4,"@maiconkusterk sua namoradaa quebrar seu ps5, ...",0,NaN


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

##### Para poder classificaf os tweets como não relevantes, relevantes ou muito relevantes, foi utilizado um conceito dado por Dale Carnegie que explica apenas criticas não são uteis para se obter melhora. Com isso nossa categoria para não relevante foram as criticas negativas, para relevante foram as criticas positivas e para o muito relevante foram comparações com concorrentes

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Analize dos dados


In [81]:
train.Classificacao.value_counts()
#161 pra cada

1    343
2    124
0    103
Name: Classificacao, dtype: int64

In [83]:
test.classificacao2.value_counts()

1    163
0    105
2     43
Name: classificacao2, dtype: int64

In [63]:
#list(train['Treinamento'])


In [77]:
train0 = train.loc[(train['Classificacao'] == 0),:]
train1 = train.loc[(train['Classificacao'] == 1),:]
train2 = train.loc[(train['Classificacao'] == 2),:]

l0 = list(train0['Treinamento'])
l1 = list(train1['Treinamento'])
l2 = list(train2['Treinamento'])

a=0
for i in l0:
    for c in i:
        a+=1         #Contador de palavras l0
        

b=0
for a in l1:
    for f in a:
        b+=1          #Contador de palavras l0
        
c=0
for a in l2:
    for f in a:
        c+=1          #Contador de palavras l0

print(a)
print(b)
print(c)

TypeError: 'float' object is not iterable

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**